In [1]:
import math
import numpy as np
import os
import pandas as pd
import re

# Some Reference

* Regular Expression Tester: https://regex101.com/
* Some Re Tutorials: https://medium.com/factory-mind/regex-tutorial-a-simple-cheatsheet-by-examples-649dc1c3f285

In [2]:
print(os.getcwd())

D:\Windows_Storage\Storage\Github\KnowledgeGraph\scripts


In [3]:
# read in tokens

# dataToken = pd.read_csv("../results/210330_result/210330_dataset_spaCyResult.csv", encoding="utf8")
dataToken = pd.read_csv("../results/210330_result/210330_dataset_monpaResult.csv", encoding="utf8")

# append a column for data if dependencies column does not exist (monpa case)
if len(dataToken.columns) < 3:
    dataToken.loc[:, "Dependecies"] = np.nan

print(len(dataToken))
print(dataToken.head(5))

114798
  Segmented Element                  POS  Dependecies
0            安森美半導體                  NaN          NaN
1                 （  PARENTHESISCATEGORY          NaN
2  ON Semiconductor                   FW          NaN
3                 ）  PARENTHESISCATEGORY          NaN
4                 ，        COMMACATEGORY          NaN


In [4]:
# MONPA
# relation_dependencies_possible_list = []
# relation_pos_possible_list = ["VH", "VC", "VJ", "VA"]
# relation_pos_re = "^[V]"
# entity_dependencies_possible_list = []
# entity_pos_possible_list = ["Na", "Nv", "Neu", "Nes", "Nf", "Ng", "Nh", "Neqa", "Nep", "Ncd", "FW", "DE"]
# entity_pos_re = "^[N]"
# bracket_pos_list = ["PARENTHESISCATEGORY"]
# punct_pos_list = ["COMMACATEGORY", "PERIODCATEGORY"]

# spaCy
relation_dependencies_possible_list = ["ROOT", "nmod:prep", "prep", "agent", ]
relation_pos_possible_list = ["VERB"]
relation_pos_re = "^[V]"
entity_dependencies_possible_list = ["compound:nn", "nsubj", "dep", "dobj"]
entity_pos_possible_list = ["NOUN", "PROPN"]
entity_pos_re = "^[N]"
bracket_pos_list = []
punct_pos_list = ["PUNCT"]

# Common, usually entity
sentences_splitter = ["，", "。", "！", "!", "？", "?"]
bracket_entity_list = ["(", ")", "（", "）"]
punct_entity_list = [" " * i for i in range(0, 100)]
conjuction_entity_list = ["的", "、", "之", "及", "與"]
not_entity_relation_list = ["的", "、", "之", "及", "與", "\r\n \r\n ", "\r\n \r\n  "] +\
[" " * i for i in range(0, 100)] + ["\n" * i for i in range(0, 100)]

In [5]:
# devide token by seperators
total_entity_list = []
sentence_entity_list = []
total_label_list = []
sentence_label_list = []
total_dependencies_list = []
sentence_dependencies_list = []


# devide tokens by chinese punctuations
for index, entityElement in enumerate(dataToken.iloc[:, 0]):
    if entityElement not in sentences_splitter:
        sentence_entity_list.append(entityElement)
        sentence_label_list.append(dataToken.iloc[index, 1])
        sentence_dependencies_list.append(dataToken.iloc[index, 2])
    else:
        total_entity_list.append(sentence_entity_list)
        total_label_list.append(sentence_label_list)
        total_dependencies_list.append(sentence_dependencies_list)
        sentence_entity_list = []
        sentence_label_list = []
        sentence_dependencies_list = []
    
print("list length (Sentences): ", len(total_entity_list), len(total_label_list), len(total_dependencies_list))


# Clean up tokens and eliminate unneccesary tokens
for sentenceIndex, sentences in enumerate(total_entity_list):
    indexPunct_0 = -9999
    indexPunct_1 = -9999
    for tokenIndex, tokens in enumerate(sentences):
        # set up element indexes that are between two punctuation
        if total_label_list[sentenceIndex][tokenIndex] in bracket_pos_list or sentences[tokenIndex] in bracket_entity_list:
            if indexPunct_0 == 0:
                indexPunct_0 = tokenIndex
            else:
                indexPunct_1 = tokenIndex
        elif total_label_list[sentenceIndex][tokenIndex] in punct_pos_list or sentences[tokenIndex] in punct_entity_list:
            # set token that fit certain POS type to ""
            total_entity_list[sentenceIndex][tokenIndex] = ""
            
    # remove tokens that are between two punctuations, and certain POS type
    if indexPunct_1 == -9999 and indexPunct_0 != -9999:
        total_entity_list[sentenceIndex][indexPunct_0] = ""
        total_label_list[sentenceIndex][indexPunct_0] = ""
        total_dependencies_list[sentenceIndex][indexPunct_0] = ""
    elif indexPunct_1 != -9999 and indexPunct_0 != -9999:
        for removeIndex in range(indexPunct_0, indexPunct_1 + 1):
            total_entity_list[sentenceIndex][removeIndex] = ""
            total_label_list[sentenceIndex][removeIndex] = ""
            total_dependencies_list[sentenceIndex][removeIndex] = ""
            
    total_entity_list[sentenceIndex] = list(filter(("").__ne__, total_entity_list[sentenceIndex]))
    total_label_list[sentenceIndex]  = list(filter(("").__ne__, total_label_list[sentenceIndex]))
    total_dependencies_list[sentenceIndex] = list(filter(("").__ne__, total_dependencies_list[sentenceIndex]))
                
print(total_entity_list[0] , total_label_list[0], total_dependencies_list[0])
print("list length: ", len(total_entity_list), len(total_label_list), len(total_dependencies_list))

list length (Sentences):  7453 7453 7453
['安森美半導體', '（', 'ON Semiconductor', '）'] [nan, 'PARENTHESISCATEGORY', 'FW', 'PARENTHESISCATEGORY'] [nan, nan, nan, nan]
list length:  7453 7453 7453


In [6]:
# get entity pairs and relations

# get relations
all_relations_list = []
all_relations_index = []
all_element_flatten = []
all_relation_flatten = []

for sentenceIndex, sentences in enumerate(total_entity_list):
    relation_list = []
    relation_index_list = []
    for tokenIndex, token in enumerate(sentences):
        if len(sentences) > 1: 
            if total_dependencies_list[sentenceIndex][tokenIndex] in relation_dependencies_possible_list or\
            total_label_list[sentenceIndex][tokenIndex] in relation_pos_possible_list or\
            re.match(relation_pos_re, str(total_label_list[sentenceIndex][tokenIndex]), flags=re.IGNORECASE) != None:
                relation_list.append(token)
                relation_index_list.append(tokenIndex)
    all_relations_list.append(relation_list)
    all_relations_index.append(relation_index_list)


# get entities
all_reformatted_entities = []
all_reformatted_index_list = []
for sentenceIndex, sentences in enumerate(total_entity_list):
    entity_index_list = []
    possible_entities = []
    reformatted_entities = []
    reformatted_index_list = []
    for tokenIndex, token in enumerate(sentences):
        if len(sentences) > 1:
            if total_dependencies_list[sentenceIndex][tokenIndex] in entity_dependencies_possible_list or\
            total_label_list[sentenceIndex][tokenIndex] in entity_pos_possible_list or\
            re.match(entity_pos_re, str(total_label_list[sentenceIndex][tokenIndex]), flags=re.IGNORECASE) != None or\
            token in conjuction_entity_list:
                entity_index_list.append(tokenIndex)
                possible_entities.append(token)
    # combine token if situate next to each other
#     print(entity_index_list, possible_entities)
    if len(possible_entities) > 0:
        combine_entity_name = possible_entities[0]
        for possibleIndex, possibleElement in enumerate(entity_index_list):
#             print(possible_entities)
            
            isContinuous = False
            if possibleIndex != 0:
                if possibleElement == entity_index_list[possibleIndex - 1] + 1:
                    isContinuous = True
                    combine_entity_name += possible_entities[possibleIndex]
                else:
                    isContinuous = False
            
                if isContinuous == False:
                    reformatted_entities.append(combine_entity_name)
                    reformatted_index_list.append(entity_index_list[possibleIndex-1])
                    combine_entity_name = possible_entities[possibleIndex]
                    
            if possibleIndex == (len(entity_index_list) - 1):
                reformatted_entities.append(combine_entity_name)
                reformatted_index_list.append(possibleElement)
                
#             print(combine_entity_name)     
#     print(reformatted_entities)
    all_reformatted_entities.append(reformatted_entities)
    all_reformatted_index_list.append(reformatted_index_list)
    
# flatten section
# remove unneccessary conjuction words
for index, element in enumerate(all_reformatted_entities):
    
    for elementIndex, elementSingle in enumerate(element):
        # remove single conjuction char in first or last position
        if elementSingle[0] in conjuction_entity_list:
            element[elementIndex] = elementSingle[1:]
        elif elementSingle[-1] in conjuction_entity_list:
            element[elementIndex] = elementSingle[:-1]
            
        all_element_flatten.append(element[elementIndex])
        
    for relationIndex, relationSingle in enumerate(all_relations_list[index]):
        # remove single conjuction char in first or last position
        if relationSingle[0] in conjuction_entity_list:
            all_relations_list[index][relationIndex] = relationSingle[1:]
        elif relationSingle[-1] in conjuction_entity_list:
            all_relations_list[index][relationIndex] = relationSingle[:-1]
            
        all_relation_flatten.append(all_relations_list[index][relationIndex])
                      
        
# print(all_relations_list, len(all_relations_list), "\n")
# print(all_reformatted_entities, len(all_reformatted_entities), )
print(len(all_relations_list), len(all_relations_index), len(all_reformatted_entities), len(all_reformatted_index_list))

# print out parsing result
for index, element in enumerate(all_reformatted_entities[:15]):
    print(element, all_reformatted_index_list[index], all_relations_list[index], all_relations_index[index])
    
data_result = pd.DataFrame({
    "Entity":all_reformatted_entities,
    "EntityIndex":all_reformatted_index_list,
    "Relation":all_relations_list,
    "RelationIndex":all_relations_index
})

7453 7453 7453 7453
['安森美半導體'] [0] [] []
['今天', '', '系列矽光電倍增管'] [0, 3, 6] ['發表', '新'] [1, 2]
['光學雷達', '擴展到其', ''] [2, 9, 11] ['廣泛', '智慧感測方案'] [10, 12]
['ArrayRDM-0112A20-QFN', '市場上首款', '車規', '產品'] [0, 5, 8, 10] ['符合'] [6]
['汽車產業及其他領域', '應用中', '需求'] [6, 9, 13] ['滿足', '增長'] [1, 11]
['ArrayRDM-0112A20-QFN', '單片1', '12', '陣列'] [0, 3, 5, 8] [] []
['安森美半導體', '市場的RDM製程'] [1, 5] ['領先'] [2]
['外 ', ''] [5, 10] ['實現', '近', '紅', '靈敏度'] [1, 3, 4, 12]
['905奈米', '業界的18.5％的光子偵測效率'] [3, 17] ['達到', '領先'] [8, 9]
['', '內部增益', '其靈敏度', '光子水準'] [1, 4, 7, 12] ['高', '使', '達到'] [2, 5, 9]
['功能', '結合使用'] [2, 6] ['高'] [3]
['', '信號'] [4, 6] ['檢測', '微弱', '返回'] [1, 3, 5]
[] [] [] []
['反射目標'] [4] ['低'] [2]
['距離'] [6] ['偵測到', '遠'] [2, 4]


In [7]:
# conculde dataframe with all tokens
        
for notIndex, notElement in enumerate(not_entity_relation_list):
    all_element_flatten = list(filter((notElement).__ne__, all_element_flatten))
    all_relation_flatten = list(filter((notElement).__ne__, all_relation_flatten))
        
data_entity = pd.DataFrame({
    "Entity":all_element_flatten,
})

data_relation = pd.DataFrame({
    "Relation":all_relation_flatten,
})



In [8]:
# import stopwords list

stopword_list = []

for fileIndex, fileElement in enumerate(os.listdir("./stopwords/")):
    if fileElement[-2:] != "md":
        data_temp = pd.read_csv("./stopwords/" + fileElement, encoding="utf8", sep="@#$%&*")
        stopword_list += data_temp.iloc[:, 0].tolist()
        
stopword_list = list(dict.fromkeys(stopword_list))
        
print(len(stopword_list), "\n\n", stopword_list[-50:])     

2310 

 ['行动', '表明', '表示', '要求', '规定', '觉得', '认为', '认真', '认识', '说明', '转动', '转变', '转贴', '达到', '迅速', '过去', '过来', '运用', '这点', '这种', '这麽', '进入', '进步', '进行', '适应', '适当', '适用', '逐渐', '通常', '造成', '遇到', '遭到', '避免', '那麽', '部分', '采取', '里面', '重大', '重新', '重要', '问题', '防止', '附近', '限制', '随著', '集中', '需要', '高兴', '是不是', '说说']


<ipython-input-8-c879b4674082>:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data_temp = pd.read_csv("../docs/stopwords/" + fileElement, encoding="utf8", sep="@#$%&*")


In [9]:
# remove stop words

data_list = [data_entity, data_relation]

for dataIndex, dataElement in enumerate(data_list):
    drop_index = []
    for rowIndex, rowElement in enumerate(dataElement.iloc[:, 0]):
        if rowElement in stopword_list:
            drop_index.append(rowIndex)
    # drop entity/relations that are inside stopword list
    print(len(dataElement))
    dataElement.drop(index=drop_index, inplace=True)
    print(len(dataElement))
    

21084
18727
14301
11894


In [10]:
# conculde a list with frequency

def ratiolize(x):
    x = str(np.around(x * 100, decimals=2)) + "%"
    return x

data_entity_value_count = data_entity.value_counts(ascending=False).to_frame()
data_entity_value_count.reset_index(inplace=True)
data_entity_value_count = data_entity_value_count.rename(columns = {'index':'Entity'})
data_entity_value_count = data_entity_value_count.rename(columns = {0:'Count'})
data_entity_value_count.loc[:, "Ratio"] = data_entity_value_count.iloc[:, 1] / len(data_entity_value_count)
data_entity_value_count.loc[:, "Ratio"] = data_entity_value_count.loc[:, "Ratio"].apply(ratiolize)

data_relation_value_count = data_relation.value_counts(ascending=False).to_frame()
data_relation_value_count.reset_index(inplace=True)
data_relation_value_count = data_relation_value_count.rename(columns = {'index':'Relation'})
data_relation_value_count = data_relation_value_count.rename(columns = {0:'Count'})
data_relation_value_count.loc[:, "Ratio"] = data_relation_value_count.iloc[:, 1] / len(data_relation_value_count)
data_relation_value_count.loc[:, "Ratio"] = data_relation_value_count.loc[:, "Ratio"].apply(ratiolize)

print(len(data_entity), data_entity_value_count[:20], "\n")
print(len(data_relation), data_relation_value_count[:20])

18727    Entity  Count  Ratio
0      晶圓    110  0.95%
1      技術    108  0.93%
2       圖     99  0.85%
3      製程     98  0.85%
4       後     87  0.75%
5       年     72  0.62%
6      晶片     63  0.54%
7      產品     61  0.53%
8       %     58   0.5%
9      我們     55  0.47%
10     設備     53  0.46%
11      中     48  0.41%
12     市場     48  0.41%
13      時     46   0.4%
14      為     45  0.39%
15    半導體     44  0.38%
16     資源     42  0.36%
17     蝕刻     41  0.35%
18      ~     39  0.34%
19     表面     39  0.34% 

11894    Relation  Count  Ratio
0         為    238  6.83%
1         高    153  4.39%
2        回收    124  3.56%
3        製造    110  3.16%
4         化    109  3.13%
5        利用    101   2.9%
6        包括     91  2.61%
7        產生     89  2.55%
8        進行     88  2.53%
9         來     72  2.07%
10       進行     70  2.01%
11       不同     65  1.87%
12       提供     64  1.84%
13        新     63  1.81%
14       萃取     61  1.75%
15       清洗     61  1.75%
16       增加     60  1.72%
17       相關   

In [11]:
# Savinf result without filtering

# data_entity_value_count.to_csv("../results/210330_result/210330_dataset_entity_result_spaCy.csv", encoding="utf8", index=None)
# data_relation_value_count.to_csv("../results/210330_result/210330_dataset_relation_result_spaCy.csv", encoding="utf8", index=None)
# data_result.to_csv("../results/210330_result/210330_dataset_main_result_spaCy.csv", encoding="utf8", index=None)

data_entity_value_count.to_csv("../results/210330_result/210330_dataset_entity_result_MONPA.csv", encoding="utf8", index=None)
data_relation_value_count.to_csv("../results/210330_result/210330_dataset_relation_result_MONPA.csv", encoding="utf8", index=None)
data_result.to_csv("../results/210330_result/210330_dataset_main_result_MONPA.csv", encoding="utf8", index=None)

# Filter out most frequent edge and relation

In [12]:
# data_entity_value_count = pd.read_csv("../results/210330_result/210330_dataset_entity_result_spaCy.csv", encoding="utf8")
# data_relation_value_count = pd.read_csv("../results/210330_result/210330_dataset_relation_result_spaCy.csv", encoding="utf8")
# data_result = pd.read_csv("../results/210330_result/210330_dataset_main_result_spaCy.csv", encoding="utf8")

data_entity_value_count = pd.read_csv("../results/210330_result/210330_dataset_entity_result_MONPA.csv", encoding="utf8")
data_relation_value_count = pd.read_csv("../results/210330_result/210330_dataset_relation_result_MONPA.csv", encoding="utf8")
data_result = pd.read_csv("../results/210330_result/210330_dataset_main_result_MONPA.csv", encoding="utf8")

# take only first 20 places
data_entity_filter = data_entity_value_count.loc[:, "Entity"].tolist()[:20]
data_relation_filter = data_relation_value_count.loc[:, "Relation"].tolist()[:20]

abandonIndexList = []

for rowIndex, rowItem in enumerate(data_result.iloc[:, 0]):
    need_to_abandon = True
    entityList = rowItem.replace("'", "").replace(" ", "").replace("[", "").replace("]", "").split(",")[:-1]
    relationList = data_result.iloc[rowIndex, 2].replace("'", "").replace(" ", "").replace("[", "").replace("]", "").split(",")[:-1]
    
    for entityIndex, entityElement in enumerate(entityList):
        if entityElement in data_entity_filter:
            need_to_abandon = False
            break
    
    for relationIndex, relationElement in enumerate(relationList):
        if need_to_abandon == False:
            break
        if relationElement in data_relation_filter:
            need_to_abandon = False
            break
            
    if need_to_abandon == True:
        abandonIndexList.append(rowIndex)
            
data_result.drop(index=abandonIndexList, inplace=True)
print(data_result)

# export data
data_result.to_csv("../results/210330_result/210330_dataset_filtered_result_spaCy.csv", encoding="utf8", index=None)

                                      Entity          EntityIndex  \
9               ['', '內部增益', '其靈敏度', '光子水準']        [1, 4, 7, 12]   
20          ['陽光條件下', '距離', '時', '訊噪', '性能']  [7, 10, 13, 18, 20]   
21               ['其他優勢', '電源偏置', '溫度變化敏感性']           [1, 7, 14]   
25                        ['應用', '市場', '方案']            [2, 6, 8]   
29                              ['其他感知模式互補']                  [5]   
...                                      ...                  ...   
7440                 ['冷凝處理', '廢氣成分中', '溫度']            [1, 6, 9]   
7441  ['冷凝作用', '兩種', '定溫', '增加系統之壓力', '在定壓']    [1, 5, 8, 13, 16]   
7444              ['冷凝器型式', '表面式及接觸式', '凝器']           [4, 9, 11]   
7447                     ['下', '蒸 氣附著', '上']            [1, 6, 9]   
7450                             ['後', '處置']               [2, 6]   

                               Relation          RelationIndex  
9                      ['高', '使', '達到']              [2, 5, 9]  
20    ['明亮', '進行', '長', '測', '提供', '佳']  